# STEEL SLITTING OPTIMA

### I Problem Statement

The Cutting Stock Problem deals with the problem of cutting stock material with the same, fixed width — such as paper rolls — into smaller pieces, according to a set of orders specifying both the widths and the demand requirements, so as to minimize the amount of wasted material.

![image](cutting_stock.png)

-----

To choose the *mother coil* from the stock to cut into order in PO received from clients. To minimize the trim loss from slitting the coil

**Key terms**
Need_cut = weight \\
Remain_Weight 
Trim_Loss_Weight 
Trim_Loss 
Num_Split


```python
if Thickness <= 3 and Thickness > 1:
        min_loss = 8
    elif Thickness == 1:
        min_loss = 5.5
    else:
        min_loss = 3*Thickness      
    max_loss= int(0.04*Width)

# Hyperpara
stock_name = 'HSC'	
Thickness = 2.6
Spec =  'JSH270C-PO' 

# Optimiza Parameter'
optima_ratio_fc1 = 50%
# optima_ratio_fc1 = 200%

In [2]:
import pandas as pd
import numpy as np

# IMPORT SAMPLE DATA
sample_po = pd.read_pickle("data/sample_po.pkl")
sample_coil = pd.read_pickle("data/sample_coils_instock.pkl")

# sample_po['Width'] = sample_po['Width'].astype('int')
# df = sample_po.groupby('Width').size().reset_index(name='Count')
# df

In [11]:
sample_po.columns

Index(['Customer', 'Description ', 'Maker', 'Spec_Name', 'Thickness', 'Width',
       'Length', 'MC Width', 'Stock_Name', 'Stock_Note', 'Need_Cut', 'T10/23',
       'T11/23', 'T12/23', 'Order_ID', 'Upper_bound_cut_w'],
      dtype='object')

In [14]:
df = sample_po[['Order_ID',"Width","Need_Cut",'T10/23']]
filtered_df = df[df['Need_Cut'] < -100]
filtered_df

,Order_ID,Width,Need_Cut,T10/23
0,0,220.0,-4247.0,22574.031480
1,1,145.0,-1895.0,4559.920560
2,2,106.0,-1414.0,10417.059900
3,3,150.0,-788.0,4618.885050
4,4,70.0,-515.0,669.131645
5,5,52.0,-400.0,0.000000
6,6,72.0,-200.0,912.258967


In [15]:
filtered_df['Upper_bound_cut_w'] = 0.5*filtered_df['T10/23'] - filtered_df["Need_Cut"]
filtered_df['Upper_bound_cut_w']

/var/folders/60/xqht0mwj245053m4bbttqr4h0000gn/T/ipykernel_285/974114791.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Upper_bound_cut_w'] = 0.5*filtered_df['T10/23']-filtered_df["Need_Cut"]


0    15534.015740
1     4174.960280
2     6622.529950
3     3097.442525
4      849.565822
5      400.000000
6      656.129483
Name: Upper_bound_cut_w, dtype: float64

In [3]:
sample_coil

,Inventory_ID,Spec_Name,Thickness,Width,Length,Weigth
0,TP235H001943,JSH270C-PO,2.6,1219,0,400.0
1,HTV1363/23,JSH270C-PO,2.6,1219,0,4000.0
2,HTV0828/23-D1,JSH270C-PO,2.6,1108,0,4266.0
3,HTV0895/23,JSH270C-PO,2.6,1219,0,5350.0


In [ ]:

# Convert DataFrame to dictionary
orders = df.to_dict(orient='index')

# Remove column names from values
for key, value in orders.items():
    orders[key] = list(value.values())
orders

## Problem Formulation

Consider a set ${S}$ of available stock materials that can be cut to size to produce a set of finished parts. Each stock $s\in S$ is characterized by a width $wd^S_s$, a weight $wt_s$ per piece, and is available in *limited* quantity. A customer order is received to product a set of finished products $F$. Each finished product $f\in F$ is specified by a required weight $wt_f$ and width $wd^F_f$.

The **cutting stock problem** is to find a minimum waste to fulfill the customer order from the stock materials.

**Stocks**

| stocks <br> $s$ | width <br> $wd^S_s$ | weight <br> $wt_s$ |
| :--: | :--: | :--: |
| A | 5 | 6 |
| B | 6 | 7 |
| C | 9 |10 |

**Finished Parts**

| finished parts <br> $f$ | width <br> $wd^F_f$ | weight <br> $wt_f$ |
| :--: | :--: | :--: |
| S | 2 | 20 |
| M | 3 | 10 |
| L | 4 | 20 |


In [2]:
stocks = {
    "A": {"Width": 1219, "weight": 4000},
    "B": {"Width": 1219, "weight": 400},
    "C": {"Width": 1108, "weight": 4266},
    "D": {"Width": 1219, "weight": 5350},
}

finish = {
    "XXL": {"Width": 220, "need_cut": -4247 , "fc1":22574},
    "L": {"Width": 145, "need_cut": -1895, "fc1": 4559},
    "XL": {"Width": 150, "need_cut": -788, "fc1": 4618},
    "M": {"Width": 70, "need_cut": -515, "fc1": 669},
    "S": {"Width": 52, "need_cut": -400, "fc1": 0},
}

## Patterns - Problem Space

One approach to solving this problem is to create a list of all finished parts, a list of stocks for each length, and then use a set of binary decision variables to assign each finished product to a particular piece of stock. This approach will work well for a small problems, but the computational complexity scales much too rapidly with the size of the problem to be practical for business applications.

A pattern is a list of finished parts that can be cut from a particular stock item.

A pattern $p$ is specified by the stock $s_p$ assigned to the pattern and integers $cut_{pf}$ that specify how many finished parts of type $f$ are cut from stock $s_p$. A pattern $p\in P$ is feasible if

$$
\begin{align}
\sum_{f\in F}cut_{pf}wd^F_f  & \leq   wd^S_{s_p}
\end{align}
$$

The function `make_patterns` defined below produces a partial list of feasible patterns for given sets of stocks and finished parts. Each pattern is represented as dictionary that specifies an associated stock item, and a dictionary of cuts that specify the finished parts cut from the stock. The algorithm is simple, it just considers every finished parts and stock items, then reports the number of parts $f$ that can be cut from stock item $s$.

A mathematical optimization model has five components, namely:

- Sets and indices.
- Parameters.
- Decision variables.
- Objective function(s).
- Constraints.

## Optimal cutting using known patterns

Given a list of patterns, the optimization problem is to compute how many copies of each pattern should be cut to meet the demand for finished parts at minimum cost.

Let the index $s_p$ denote the stock specified by pattern $p$, and let $x_{s_p}$ denote the number pieces of stock $s_p$ is used. For a given list of patterns, the minimum cost optimization problem is a mixed integer linear optimization (MILO) subject to meeting demand constraints for each finished item.

### Sets and Indices

$s \in \text{Stock}=\{1,2,\dots, N\}$: Set of available stock material units.

$f \in \text{Finished Good}$: Set of final finised goods.

### Decision variables

$ \text{trimloss}_i \in [x \text{ mm}, 4\% \text{ Mother\_Coil\_Width}]:  x (mm) $ the lower bound, depending on the thickness of the Mother Coil if stock $i$ is used; $4\%$ the upper bound .

$\text{cut}_{i,j} \in \mathbb{N}$: Number of pieces of type $j$ cut out from stock material unit $i$.

### Objective Functions

- **Material Consumption:** Minimize the number of stock material units used.

$$
\begin{equation}
\text{Min} \sum_{i \in \text{Stock}}{\text{trimloss}_i}
\tag{1.0}
\end{equation}
$$

### Constraints


#### Weight Constraints
- **Satisfy Demand:** Total weight of final pieces of type $j$ obtained must satisfy its demand requirement.

$$
\begin{equation}
\text{Upper bound weight}_j \geq P{p \in \text{Patterns}}{\text{ Cut}_{p,j} \cdot \text{Weight per cut}_p}   \quad \forall j \in \text{Finished Goods}
\tag{2.1}
\end{equation}
$$



The following cell is an AMPL implementation of this optimization model.

In [21]:
from gurobipy import multidict

number_of_vehicles = 2 
vehicles_origin = [4, 6]
vehicles_destination = [3, 0]
# total_time_vehicle = [35, 27]

truck = [i for i in range(number_of_vehicles)]
starting_node = {}
destination_nodes = {}
time = {}
for i in truck:
  starting_node[i] = vehicles_origin[i]
  destination_nodes[i] = vehicles_destination[i]
#   time[i] = total_time_vehicle[i]


multi = {}
for i in range(number_of_vehicles):
  l = [vehicles_origin[i],vehicles_destination[i],
    #    total_time_vehicle[i]
       ]
  multi[i] = l

truck, starting_node, destination_nodes = multidict(multi)

In [22]:
multi

{0: [4, 3], 1: [6, 0]}

In [23]:
piece_reqs = [length*req for length, req in multi.values()]